In [ ]:
%pip install faster-whisper gradio imageio[ffmpeg] yt-dlp langchain langchain-ollama langchain-community

In [ ]:
# Check if ffmpeg is installed and where it is located
import imageio_ffmpeg
print(imageio_ffmpeg.get_ffmpeg_exe())

In [ ]:
from tqdm import tqdm
import textwrap
from faster_whisper import WhisperModel

In [ ]:
!yt-dlp -f "bestaudio" -x --audio-format mp3 -o "video_audio.mp3" --ffmpeg-location "your/local/path/to/ffmpeg" "https://www.youtube.com/watch?v=cXL-E7I6Oe4"

[youtube] Extracting URL: https://www.youtube.com/watch?v=cXL-E7I6Oe4
[youtube] cXL-E7I6Oe4: Downloading webpage
[youtube] cXL-E7I6Oe4: Downloading tv client config
[youtube] cXL-E7I6Oe4: Downloading player 6b3caec8
[youtube] cXL-E7I6Oe4: Downloading tv player API JSON
[youtube] cXL-E7I6Oe4: Downloading ios player API JSON
[youtube] cXL-E7I6Oe4: Downloading m3u8 information
[info] cXL-E7I6Oe4: Downloading 1 format(s): 251
[download] Destination: video_audio.webm

[download]   0.0% of    2.78MiB at  499.86KiB/s ETA 00:05
[download]   0.1% of    2.78MiB at    1.46MiB/s ETA 00:01
[download]   0.2% of    2.78MiB at    3.42MiB/s ETA 00:00
[download]   0.5% of    2.78MiB at    7.32MiB/s ETA 00:00
[download]   1.1% of    2.78MiB at    5.50MiB/s ETA 00:00
[download]   2.2% of    2.78MiB at    5.77MiB/s ETA 00:00
[download]   4.5% of    2.78MiB at    7.64MiB/s ETA 00:00
[download]   9.0% of    2.78MiB at   11.73MiB/s ETA 00:00
[download]  17.9% of    2.78MiB at   18.23MiB/s ETA 00:00
[download]

In [33]:
def wrap_paragraphs(text, width=80) -> str:
    """
    Wrap paragraphs of text to a given width.
    
    Args:
        text: The text to wrap.
        width: The width to wrap the text to.

    Returns:    
        The wrapped text.
    """
    paragraphs = text.split('\n\n')  # split on double newlines
    wrapped_paragraphs = []
    for paragraph in paragraphs:
        wrapped_paragraphs.append(textwrap.fill(paragraph, width=width))
    return "\n\n".join(wrapped_paragraphs)

In [11]:
model_size = "small"

model = WhisperModel(model_size, device="cpu", compute_type="int8")

In [ ]:
def transcribe(audio_file) -> str:
    """
    Transcribe an audio file.

    Args:
        audio_file: The path to the audio file to transcribe.

    Returns:
        The transcription of the audio file.
    """

    segments, info = model.transcribe(audio_file, beam_size=5)
    print("Detected language '%s' with probability %f" % (info.language, info.language_probability))
    
    transcription = ""
    for segment in tqdm(segments, desc="Transcribing", unit="segment"):
        transcription += "[%.2fs -> %.2fs] %s\n" % (segment.start, segment.end, segment.text)
    
    return transcription

In [25]:
transcription = transcribe("video_audio.mp3")

Detected language 'pt' with probability 0.981688


Transcribing: 41segment [00:25,  1.59segment/s]


In [30]:
print(transcription)

[0.00s -> 7.28s]  O 360 está de volta e agora vamos fazer um giro de notícias com os nossos
[7.28s -> 11.70s]  repórteres espalhados pelo Brasil? Vamos começar, olha, já temos Luciana
[11.70s -> 17.60s]  Maral aqui de Brasília. Brasília Camila Tícia em Salvador e Adriana de
[17.60s -> 22.88s]  Lucca em São Paulo. Vamos começar com a Luciana Maral, porque aqui de Brasília,
[22.88s -> 27.60s]  porque é a Comissão Temporária do Senado, que trata da regulamentação da
[27.60s -> 33.68s]  inteligência artificial, aprovou hoje a proposta que cria regras para o uso dessa
[33.68s -> 38.88s]  tecnologia no Brasil. Luciana, é uma informação muito importante, porque
[38.88s -> 45.92s]  traça, né? Um risco no chão, porque por enquanto, do jeito que está, está muito
[45.92s -> 50.40s]  o Adeus Dará. Conta pra gente então o que foi aprovado e qual que é a
[50.40s -> 55.68s]  tremitação daqui pra frente. Boa tarde pra todas vocês, meninas.
[57.60s -> 63.24s]  Tudo bem, Debra? Muito boa tarde a você, a

In [31]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# Example of an LLM
llm = ChatOllama(temperature=0, model="gemma3:12b")

template = """You are an expert assistant in refining raw video transcriptions. The text provided contains timestamps, occasional disfluencies, and formatting artifacts that make it hard to read. Your task is to reformat the transcription so that it is clear and well-organized, while preserving all the original content and details. Do not summarize or omit any information; just remove unnecessary timestamps and artifacts, and adjust the text for improved readability.

Raw Transcription:
{transcription}

Refined Transcription (in the language of the transcription):
"""

prompt = ChatPromptTemplate.from_template(template)

messages = prompt.invoke({"transcription": transcription})

response = llm.invoke(messages)

In [35]:
response

AIMessage(content='O 360 está de volta e agora vamos fazer um giro de notícias com os nossos repórteres espalhados pelo Brasil. Vamos começar. Olha, já temos Luciana Maral aqui de Brasília, Camila Tícia em Salvador e Adriana de Lucca em São Paulo. Vamos começar com a Luciana Maral, porque aqui de Brasília, a Comissão Temporária do Senado, que trata da regulamentação da inteligência artificial, aprovou hoje a proposta que cria regras para o uso dessa tecnologia no Brasil.\n\nLuciana, é uma informação muito importante, porque traça um risco no chão, porque por enquanto, do jeito que está, está muito...\n\nBoa tarde pra todas vocês, meninas. Tudo bem, Debra? Muito boa tarde a você, as meninas, a todos que nos acompanham aqui no 360, excelente tarde de quinta-feira a todo mundo. Pois é, demorou a ser aprovado a passar por essa Comissão Temporária no Senado. Levou mais de seis meses, se a gente for levar em consideração, desde quando chegou, então, ali a esse colegiado teve muita divergênci

In [36]:
wrapped_answer = wrap_paragraphs(response.content, width=100)
print(wrapped_answer)

O 360 está de volta e agora vamos fazer um giro de notícias com os nossos repórteres espalhados pelo
Brasil. Vamos começar. Olha, já temos Luciana Maral aqui de Brasília, Camila Tícia em Salvador e
Adriana de Lucca em São Paulo. Vamos começar com a Luciana Maral, porque aqui de Brasília, a
Comissão Temporária do Senado, que trata da regulamentação da inteligência artificial, aprovou hoje
a proposta que cria regras para o uso dessa tecnologia no Brasil.

Luciana, é uma informação muito importante, porque traça um risco no chão, porque por enquanto, do
jeito que está, está muito...

Boa tarde pra todas vocês, meninas. Tudo bem, Debra? Muito boa tarde a você, as meninas, a todos que
nos acompanham aqui no 360, excelente tarde de quinta-feira a todo mundo. Pois é, demorou a ser
aprovado a passar por essa Comissão Temporária no Senado. Levou mais de seis meses, se a gente for
levar em consideração, desde quando chegou, então, ali a esse colegiado teve muita divergência,
porque a oposição di